# DeadSpace
ETH to_addresses queried till block 16,000,000 and stored on GCP Storage

Install prerequisite Python packages
pip install scipy
pip install pandas

Download gzip-ed data files into ETH_addresses

In [ ]:
import time
from multiprocessing import Pool
import pandas as pd
import scipy
import os
from scipy.stats import kstest
from scipy.stats import chisquare

In [ ]:
#Chi-Squared randomness test
def chi_str(add_str: str):
  # Compute the observed frequency of each character in the string
  observed_freq = {char: add_str.count(char) for char in set(add_str)}

  # Compute the expected frequency of each character in a random string of the same length
  expected_freq = {char: len(add_str) / len(set(add_str)) for char in set(add_str)}

  # Compute the chi-squared statistic
  chi_squared = chisquare(list(observed_freq.values()), f_exp=list(expected_freq.values()))

  return chi_squared


In [ ]:
#Kolmogorov-Smirnov randomness Test
def ks_str(add_str: str):

  # Compute the cumulative probability distribution of characters in the string
  distribution = [add_str.index(char) / len(add_str) for char in add_str]

  # Compute the Kolmogorov-Smirnov statistic
  ks_statistic = kstest(distribution, 'uniform')

  return ks_statistic

In [ ]:
def etl(file: str):
  start_t = time.perf_counter()

  # Unzip file
  os.system('gunzip -k /storage/ETH_addresses/' + file + ' -c > /storage/tmp/'+file+'_out.json')
  os.system('cat /storage/tmp/'+file+'_out.json | jq .to_address > /storage/tmp/'+file+'_addresses.json')
  os.system('rm -rf /storage/tmp/'+file+'_out.json')

  # Read file into lines[]
  read_file = open('/storage/tmp/'+file+'_addresses.json', "r")
  lines = read_file.read().strip().replace('"','').replace('null\n','').split("\n")
  read_file.close()

  # PD DataFrame
  df = pd.DataFrame(columns=['Address','chi', 'chi_p','ks','ks_p'])

  # evaluate randomness of each address
  for cnt in range(len(lines)):
    chi_result = chi_str(lines[cnt])
    ks_result = ks_str(lines[cnt])
    df.loc[cnt,:] = [lines[cnt], round(chi_result.statistic,3), round(chi_result.pvalue,3), round(ks_result.statistic,3), format(ks_result.pvalue,'.3e')]

  # Save output
  df.to_csv('/storage/ETH_randomness/'+file+'.csv')

  # Clean up
  os.system('rm -rf /storage/tmp/'+file+'_addresses.json')

  end_t = time.perf_counter()

  return file, start_t - end_t

In [ ]:
# Read files to be processed
files = os.listdir('/storage/ETH_addresses/')
start_t = time.perf_counter()

# Run each file on separate CPU core
with Pool() as pool:
  results = pool.imap_unordered(etl, files)
  for filename, duration in results:
    print(f"{filename} completed in {duration:.2f}s")

ent_t = time.perf_counter()
total_duration = ent_t - start_t
print(f"files took {total_duration:.2f}s total")